# Třetí cvičení - jazykové modely, perplexita

## 1) Příprava dat

- Soubory TEXTCZ1 a TEXTEN1 přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase

In [46]:
import re, string
from collections import Counter
from math import log
import math

## Unigramový model
- Vypočítejte pravděpodobnost každého ze slov
- Vypočítejte entropii H(X) a perplexitu G(X) modelu

$H(X) = - \sum_{x}p(x)log{_2}p(x)$

$G(X) = 2^{H(X)}$

In [47]:
# Nejprve vytvořte unigramový model pro větu: "<s> Dnes je hezký den"
sent = "<s> Dnes je hezký den"
freq = Counter(sent.split())
total = freq.total()
entropy = -sum(
    (count / total) * log(count / total, 2)
    for _, count in freq.items()
)
perplexity = 2 ** entropy

# Očekávaný výsledek: 
# entropy = 2.321928094887362
# perplexity = 4.999999999999999

print(f"Entropy: {entropy} Perplexity: {perplexity}")

Entropy: 2.321928094887362 Perplexity: 4.999999999999999


In [48]:
# Nyní pro CZ a EN korpusy

In [49]:
# Načtení souborů CZ, EN
with open('TEXTCZ1.txt', "rb") as f:
    cz_text = f.read().decode('cp1250').splitlines()
with open('TEXTEN1.txt', "r") as f:
    en_text = f.read().splitlines()
        

In [50]:
# Rozdělení textů na věty, odstranění interpunkce a přidání znaku začátku věty

def sentences_unigram(text):
    split_on = set('.?!')
    sentences = [['<s>']]
    for word in text:
        if re.match(r'[\w\-]+', word):
            sentences[-1].append(word.lower())
        elif word in split_on:
            sentences.append(['<s>'])
    return sentences

def unigram(sentences):
    return Counter([
        word
        for sent in sentences
        for word in sent
    ])

In [51]:
# Anglický unigramový model

en_sent = sentences_unigram(en_text)
en_model = unigram(en_sent)
en_total = en_model.total()

en_entropy = -sum(
    (count / en_total) * log(count / en_total, 2)
    for _, count in en_model.items()
)
en_perplexity = 2 ** en_entropy

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 9.153156076444484
# en_perplexity = 569.3437191681389

Entropy: 9.15595455066942 Perplexity: 570.4491780329872


In [52]:
# Český unigramový model

cz_sent = sentences_unigram(cz_text)
cz_model = unigram(cz_sent)
cz_total = cz_model.total()

cz_entropy =  -sum(
    (count / cz_total) * log(count / cz_total, 2)
    for _, count in cz_model.items()
)
cz_perplexity = 2 ** cz_entropy

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 11.866640196670001
# en_perplexity = 3734.346796191333


Entropy: 11.834792150303567 Perplexity: 3652.8129153021596


## Bigramový model
- obdobně vytvořte bigramový model a všechny dvojice slov hledejte vždy pouze ve větě (nikoliv mezi větami)
- vypočítejte entropii a perplexitu

$H(B|A) = - \sum_{a \in A, b \in B}P(a,b)log{_2}P(b|a)$ 

$G(X) = 2^{H(B|A)}$

In [53]:
# Nejprve vytvořte bigramový model pro větu: "<s> Dnes je hezký den </s>"
sent = "<s> dnes respektive dnes možná </s>".split()
w_freq = Counter(sent)
b_freq = Counter(zip(sent[:-1], sent[1:]))
total = b_freq.total()

entropy = -sum( # If a pair isn't found, its contribution is times zero
    (count / total) * log(count / w_freq[w1], 2)
    for (w1, _), count in b_freq.items()
)
perplexity = 2 ** entropy

print(f"Entropy: {entropy} Perplexity: {perplexity}")
# Očekávaný výsledek: 
# entropy = 0.4
# perplexity = 1.3195079107728942

Entropy: 0.4 Perplexity: 1.3195079107728942


In [54]:
# Nyní pro jednotlivé korpusy, nejprve pro CZ a pak pro EN

# Přidání koncového znaku </s> do vět

def sentences_bigram(text):
    return [
        sent + ['</s>']
        for sent in sentences_unigram(text)
    ]

def bigram(sentences):
    b_freq = Counter([
        (w1, w2)
        for sent in sentences
        for (w1, w2) in zip(sent[:-1], sent[1:])
    ])
    return b_freq, unigram(sentences)

In [55]:
en_sent = sentences_bigram(en_text)
en_model, en_uni = bigram(en_sent)
en_total = en_model.total()

en_entropy = -sum(
    (count / en_total) * log(count / en_uni[w1], 2)
    for (w1, _), count in en_model.items()
)
en_perplexity = 2 ** en_entropy

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")

# Očekávaný výsledek: 
# en_entropy = 5.380173547272233
# en_perplexity = 41.64794906297528

Entropy: 5.380436055874065 Perplexity: 41.65552789238245


In [56]:
cz_sent = sentences_bigram(cz_text)
cz_model, cz_uni = bigram(cz_sent)
cz_total = cz_model.total()

cz_entropy = -sum(
    (count / cz_total) * log(count / cz_uni[w1], 2)
    for (w1, _), count in cz_model.items()
)
cz_perplexity = 2 ** cz_entropy

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")

# Očekávaný výsledek: 
# cz_entropy = 4.624183207248371
# cz_perplexity = 24.6614070105357

Entropy: 4.651925957848192 Perplexity: 25.140230250621457


In [57]:
#SRILM příkazy
#ngram-count -text TEXTEN_unigram.txt -order 2 -write czbigram.count -unk
#ngram-count -gt1max 0 -gt2max 0 -read czbigram.count -order 2 -lm czbigram.lm
#ngram -ppl TEXTEN_unigram.txt -order 2 -lm czbigram.lm >> czbigram.ppl

